<a href="https://colab.research.google.com/github/pgreisen/PEVAE_Paper/blob/master/DecipheringProteinLandscape_p450_colabversion_with_google_drive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Test the VAE model from the paper "Deciphering protein evolution and fitness landscapes with latent space models" 

In [ ]:
# mount google drive and copy files
from google.colab import drive
drive.mount('./drive')

Mounted at ./drive


In [ ]:
!cp -r "/content/drive/My Drive/Colab Notebooks/VAE/pfam_msa/script" .
!cp -r "/content/drive/My Drive/Colab Notebooks/VAE/script" .

cp: cannot stat '/content/drive/My Drive/Colab Notebooks/VAE/pfam_msa/script': No such file or directory


In [ ]:
# reinitialized working space
!rm -r output/
!rm -r MSA/
!mkdir output/
!mkdir output/model/
!mkdir MSA

rm: cannot remove 'output/': No such file or directory
rm: cannot remove 'MSA/': No such file or directory


In [ ]:
!pwd

/content


In [ ]:
import urllib3
import gzip
import sys

In [ ]:
# Insert pfam id
# P450 // TAT
pfam_id = 'PF00067'


## Download the multiple sequence alignment for a given Pfam ID

In [ ]:

import urllib3
import gzip
import sys
'''
import argparse

parser = argparse.ArgumentParser(description = "Download the full multiple sequence alignment (MSA) in Stockholm format for a Pfam_id.")
parser.add_argument("--Pfam_id", help = "the ID of Pfam; e.g. PF00041")
args = parser.parse_args()
pfam_id = args.Pfam_id'''


print("Downloading the full multiple sequence alignment for Pfam: {0} ......".format(pfam_id))
http = urllib3.PoolManager()
r = http.request('GET', 'http://pfam.xfam.org/family/{0}/alignment/full/gzipped'.format(pfam_id))
data = gzip.decompress(r.data)
data = data.decode()
with open("./MSA/{0}_full.txt".format(pfam_id), 'w') as file_handle:
    print(data, file = file_handle)



In [ ]:
# P450 A160 
#gs = "CP2CJ_HUMAN/30-487"
gs= "D5DYE1_BACMQ/6-449"

## Process MSA

In [ ]:

import pickle
import sys
import numpy as np
from sys import exit


pfam_name = pfam_id
file_name = "./MSA/"+pfam_name+"_full.txt"
query_seq_id = gs


## read all the sequences into a dictionary
seq_dict = {}
with open(file_name, 'r') as file_handle:
    for line in file_handle:
        if line[0] == "#" or line[0] == "/" or line[0] == "":
            continue
        line = line.strip()
        if len(line) == 0:
            continue
        seq_id, seq = line.split()

        seq_dict[seq_id] = seq.upper()
        
## remove gaps in the query sequences
query_seq = seq_dict[query_seq_id] ## with gaps
idx = [ s == "-" or s == "." for s in query_seq]
for k in seq_dict.keys():
    seq_dict[k] = [seq_dict[k][i] for i in range(len(seq_dict[k])) if idx[i] == False]
query_seq = seq_dict[query_seq_id] ## without gaps

with open("./output/query_seq.fasta", 'w') as file_handle:
    print("> query", file = file_handle)
    print("".join(query_seq), file = file_handle)

## remove sequences with too many gaps
len_query_seq = len(query_seq)
seq_id = list(seq_dict.keys())
num_gaps = []
for k in seq_id:
    num_gaps.append(seq_dict[k].count("-") + seq_dict[k].count("."))
    #if seq_dict[k].count("-") + seq_dict[k].count(".") > len_query_seq*0.10:
    #print("len_query_seq*0.10 = %s" % str(len_query_seq*0.10))
    if seq_dict[k].count("-") + seq_dict[k].count(".") > 30:
        seq_dict.pop(k)

with open("./output/seq_dict.pkl", 'wb') as file_handle:
    pickle.dump(seq_dict, file_handle)
        
## convert aa type into num 0-20
aa = ['R', 'H', 'K',
      'D', 'E',
      'S', 'T', 'N', 'Q',
      'C', 'G', 'P',
      'A', 'V', 'I', 'L', 'M', 'F', 'Y', 'W']
aa_index = {}
aa_index['-'] = 0
aa_index['.'] = 0
i = 1
for a in aa:
    aa_index[a] = i
    i += 1
with open("./output/" + "/aa_index.pkl", 'wb') as file_handle:
    pickle.dump(aa_index, file_handle)
    
seq_msa = []
keys_list = []
for k in seq_dict.keys():
    if seq_dict[k].count('X') > 0 or seq_dict[k].count('Z') > 0:
        continue    
    seq_msa.append([aa_index[s] for s in seq_dict[k]])
    keys_list.append(k)    
seq_msa = np.array(seq_msa)

with open("./output/keys_list.pkl", 'wb') as file_handle:
    pickle.dump(keys_list, file_handle)

## remove positions where too many sequences have gaps
pos_idx = []
for i in range(seq_msa.shape[1]):
    if np.sum(seq_msa[:,i] == 0) <= seq_msa.shape[0]*0.20:
    #if np.sum(seq_msa[:,i] == 0) <= seq_msa.shape[0]*0.20:
        pos_idx.append(i)
with open("./output/" + "/seq_pos_idx.pkl", 'wb') as file_handle:
    pickle.dump(pos_idx, file_handle)
    
seq_msa = seq_msa[:, np.array(pos_idx)]
with open("./output/" + "/seq_msa.pkl", 'wb') as file_handle:
    pickle.dump(seq_msa, file_handle)

## reweighting sequences
seq_weight = np.zeros(seq_msa.shape)
for j in range(seq_msa.shape[1]):
    aa_type, aa_counts = np.unique(seq_msa[:,j], return_counts = True)
    num_type = len(aa_type)
    aa_dict = {}
    for a in aa_type:
        aa_dict[a] = aa_counts[list(aa_type).index(a)]
    for i in range(seq_msa.shape[0]):
        seq_weight[i,j] = (1.0/num_type) * (1.0/aa_dict[seq_msa[i,j]])
tot_weight = np.sum(seq_weight)
seq_weight = seq_weight.sum(1) / tot_weight 
with open("./output/" + "/seq_weight.pkl", 'wb') as file_handle:
    pickle.dump(seq_weight, file_handle)

## change aa numbering into binary
K = 21 ## num of classes of aa
D = np.identity(K)
num_seq = seq_msa.shape[0]
len_seq_msa = seq_msa.shape[1]
seq_msa_binary = np.zeros((num_seq, len_seq_msa, K))
for i in range(num_seq):
    seq_msa_binary[i,:,:] = D[seq_msa[i]]

with open("./output/" + "/seq_msa_binary.pkl", 'wb') as file_handle:
    pickle.dump(seq_msa_binary, file_handle)


In [ ]:

import numpy as np
import pickle

with open("./output/seq_msa.pkl", 'rb') as file_handle:
    seq_msa = pickle.load(file_handle)
seq_msa = seq_msa.tolist()

seq_msa_string = []
for seq in seq_msa:    
    seq_msa_string.append(",".join([str(s) for s in seq]))
seq_msa_set = set(seq_msa_string)

print("Original num of seq: {}".format(len(seq_msa_string)))
print("Num of uniq seq: {}".format(len(seq_msa_set)))


Original num of seq: 41322
Num of uniq seq: 38727


## Training for the models

In [ ]:
# train.py
import numpy as np
import matplotlib as mpl
mpl.use('Agg')
import matplotlib.pyplot as plt
import pickle
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import sys
# path needs to be set to your system
sys.path.append("./script")
from VAE_model import *
from sys import exit
import argparse


# PG edits
num_epoches = 10000 # args.num_epoch
weight_decay = 0.01 # args.weight_decay

print("num_epoches: ", num_epoches)
print("weight_decay: ", str(weight_decay))

## read data
with open("./output/seq_msa_binary.pkl", 'rb') as file_handle:
    seq_msa_binary = pickle.load(file_handle)    
num_seq = seq_msa_binary.shape[0]
len_protein = seq_msa_binary.shape[1]
num_res_type = seq_msa_binary.shape[2]
seq_msa_binary = seq_msa_binary.reshape((num_seq, -1))
seq_msa_binary = seq_msa_binary.astype(np.float32)

with open("./output/seq_weight.pkl", 'rb') as file_handle:
    seq_weight = pickle.load(file_handle)
seq_weight = seq_weight.astype(np.float32)

with open("./output/keys_list.pkl", 'rb') as file_handle:
    seq_keys = pickle.load(file_handle)

#### training model with K-fold cross validation
## split the data index 0:num_seq-1 into K sets
## each set is just a set of indices of sequences.
## in the kth traing, the kth subsets of sequences are used
## as validation data and the remaining K-1 sets are used
## as training data
K = 5
num_seq_subset = num_seq // K + 1
idx_subset = []
random_idx = np.random.permutation(range(num_seq))
for i in range(K):
    idx_subset.append(random_idx[i*num_seq_subset:(i+1)*num_seq_subset])

## the following list holds the elbo values on validation data    
elbo_all_list = []

for k in range(K):
    print("Start the {}th fold training".format(k))
    print("-"*60)
    
    ## build a VAE model with random parameters
    vae = VAE(21, 2, len_protein * num_res_type, [100])

    ## move the VAE onto a GPU
    vae.cuda()

    ## build the Adam optimizer
    optimizer = optim.Adam(vae.parameters(),
                           weight_decay = weight_decay)

    ## collect training and valiation data indices
    validation_idx = idx_subset[k]
    validation_idx.sort()
    
    train_idx = np.array(list(set(range(num_seq)) - set(validation_idx)))
    train_idx.sort()

    train_msa = torch.from_numpy(seq_msa_binary[train_idx, ])
    train_msa = train_msa.cuda()

    train_weight = torch.from_numpy(seq_weight[train_idx])
    train_weight = train_weight/torch.sum(train_weight)
    train_weight = train_weight.cuda()
    
    train_loss_list = []    
    for epoch in range(num_epoches):    
        loss = (-1)*vae.compute_weighted_elbo(train_msa, train_weight)
        optimizer.zero_grad()
        loss.backward()        
        optimizer.step()

        train_loss_list.append(loss.item())
        if (epoch + 1) % 50 ==0:
            print("Fold: {}, Epoch: {:>4}, loss: {:>4.2f}".format(k, epoch, loss.item()), flush = True)

    ## cope trained model to cpu and save it
    vae.cpu()
    torch.save(vae.state_dict(), "./output/model/vae_{}_fold_{}.model".format(str(weight_decay), k))

    print("Finish the {}th fold training".format(k))
    print("="*60)
    print('')
    
    print("Start the {}th fold validation".format(k))
    print("-"*60)
    ## evaluate the trained model 
    vae.cuda()
    
    elbo_on_validation_data_list = []
    ## because the function vae.compute_elbo_with_multiple samples uses
    ## a large amount of memory on GPUs. we have to split validation data
    ## into batches.
    batch_size = 32
    num_batches = len(validation_idx)//batch_size + 1
    for idx_batch in range(num_batches):
        if (idx_batch + 1) % 50 == 0:
            print("idx_batch: {} out of {}".format(idx_batch, num_batches))        
        validation_msa = seq_msa_binary[validation_idx[idx_batch*batch_size:(idx_batch+1)*batch_size]]
        validation_msa = torch.from_numpy(validation_msa)
        with torch.no_grad():
            validation_msa = validation_msa.cuda()
            elbo = vae.compute_elbo_with_multiple_samples(validation_msa, 5000)            
            elbo_on_validation_data_list.append(elbo.cpu().data.numpy())

    elbo_on_validation_data = np.concatenate(elbo_on_validation_data_list)    
    elbo_all_list.append(elbo_on_validation_data)
    
    print("Finish the {}th fold validation".format(k))
    print("="*60)

    torch.cuda.empty_cache()
    
elbo_all = np.concatenate(elbo_all_list)
elbo_mean = np.mean(elbo_all)
## the mean_elbo can approximate the quanlity of the learned model
## we want a model that has high mean_elbo
## different weight decay factor or different network structure
## will give different mean_elbo values and we want to choose the
## weight decay factor or network structure that has large mean_elbo
print("mean_elbo: {:.3f}".format(elbo_mean))

with open("./output/elbo_all.pkl", 'wb') as file_handle:
    pickle.dump(elbo_all, file_handle)

## after choosing the weight decay and network structure that have
## large mean_elbo, we can fix them and train a new model with all
## the training data as before

    
#exit()


num_epoches:  10000
weight_decay:  0.01
Start the 0th fold training
------------------------------------------------------------
Fold: 0, Epoch:   49, loss: 897.52
Fold: 0, Epoch:   99, loss: 855.90
Fold: 0, Epoch:  149, loss: 834.41
Fold: 0, Epoch:  199, loss: 822.18
Fold: 0, Epoch:  249, loss: 813.78
Fold: 0, Epoch:  299, loss: 806.56
Fold: 0, Epoch:  349, loss: 799.84
Fold: 0, Epoch:  399, loss: 793.54
Fold: 0, Epoch:  449, loss: 787.59
Fold: 0, Epoch:  499, loss: 781.94
Fold: 0, Epoch:  549, loss: 777.04
Fold: 0, Epoch:  599, loss: 771.99
Fold: 0, Epoch:  649, loss: 767.68
Fold: 0, Epoch:  699, loss: 763.71
Fold: 0, Epoch:  749, loss: 759.92
Fold: 0, Epoch:  799, loss: 756.48
Fold: 0, Epoch:  849, loss: 752.82
Fold: 0, Epoch:  899, loss: 749.47
Fold: 0, Epoch:  949, loss: 746.29
Fold: 0, Epoch:  999, loss: 742.66
Fold: 0, Epoch: 1049, loss: 739.50
Fold: 0, Epoch: 1099, loss: 736.43
Fold: 0, Epoch: 1149, loss: 733.60
Fold: 0, Epoch: 1199, loss: 730.80
Fold: 0, Epoch: 1249, loss: 728

Plotting the results from the VAE of the immunoglobolin family

## Analysis the model

In [ ]:


import matplotlib as mpl
mpl.use('Agg')
import matplotlib.pyplot as plt
from torch.autograd import Variable
import numpy as np
import pickle
import torch
import pandas
# PG edits
#from model import *
import sys
sys.path.append("/content/script")
from VAE_model import *
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sys import exit

mpl.rc('font', size = 16)
mpl.rc('axes', titlesize = 'large', labelsize = 'large')
mpl.rc('xtick', labelsize = 'large')
mpl.rc('ytick', labelsize = 'large')

## read data
with open("./output/seq_msa_binary.pkl", 'rb') as file_handle:
    seq_msa_binary = pickle.load(file_handle)    
num_seq = seq_msa_binary.shape[0]
len_protein = seq_msa_binary.shape[1]
num_res_type = seq_msa_binary.shape[2]
seq_msa_binary = seq_msa_binary.reshape((num_seq, -1))
seq_msa_binary = seq_msa_binary.astype(np.float32)

with open("./output/seq_weight.pkl", 'rb') as file_handle:
    seq_weight = pickle.load(file_handle)
seq_weight = seq_weight.astype(np.float32)

batch_size = num_seq
# PG comments
#train_data = MSA_Dataset(seq_msa_binary, seq_weight)
train_data = MSA_Dataset(seq_msa_binary, seq_weight, seq_keys)
train_data_loader = DataLoader(train_data, batch_size = batch_size)
VAE(21, 2, len_protein * num_res_type, [100])
vae.cuda()
vae.load_state_dict(torch.load("./output/model/vae_0.01_fold_0.model"))

for idx, data in enumerate(train_data_loader):
    # PG comment
    #msa, weight = data
    msa, weight = data[0],data[1]
    with torch.no_grad():
        msa = Variable(msa).cuda()   
        mu, sigma, p = vae.forward(msa)

mu = mu.cpu().data.numpy()
sigma = sigma.cpu().data.numpy()
p = p.cpu().data.numpy()
with open("./output/latent_space.pkl", 'wb') as file_handle:
    pickle.dump({'mu':mu, 'sigma': sigma, 'p': p}, file_handle)

with open("./output/latent_space.pkl", 'rb') as file_handle:
    latent_space = pickle.load(file_handle)
mu = latent_space['mu']
sigma = latent_space['sigma']
p = latent_space['p']
    
plt.figure(0)
plt.clf()
plt.plot(mu[:,0], mu[:,1], '.', alpha = 0.1, markersize = 3)
plt.xlim((-6,6))
plt.ylim((-6,6))
plt.xlabel("$Z_1$")
plt.ylabel("$Z_2$")
plt.tight_layout()
plt.savefig("./output/immuno_globolins_latent_mu_scatter.png")


## project latent space to sequence

In [ ]:


import numpy
import torch
import sys
sys.path.append("/content/script")
from VAE_model import *
import pickle
import argparse
from sys import exit

## read training to get the length of protein and the number of
## amino acide types
with open("./output/seq_msa_binary.pkl", 'rb') as file_handle:
    seq_msa_binary = pickle.load(file_handle)
len_protein = seq_msa_binary.shape[1]
num_aa_type = seq_msa_binary.shape[2]

## load trained VAE model    
vae = VAE(21, 2, len_protein * num_aa_type, [100])
model_state_dict = torch.load("./output/model/vae_0.01_fold_0.model")
vae.load_state_dict(model_state_dict)

## define points in latent space to be converted into sequences
## here I will just use random points as an example
num_seqs = 5
dim_latent_space = 2
z = torch.randn(num_seqs, dim_latent_space)
## z has to be a torch.tensor with a size of (num_seqs, dim_latent_space)
with torch.no_grad():
    log_p = vae.decoder(z)
    p = torch.exp(log_p)
    p = torch.reshape(p, (num_seqs, len_protein, num_aa_type))
p = p.numpy()

## p is a numpy array with shape [num_seqs, len_protein, num_aa_type].
## p[i,:,:] represents the sequence converted from the latent space point z[i,:].
## p[i,:,:] represents the probabilities of each amino acid at all positions of the i'th sequence.
## Therefore, each point in the latent space is converted into a distribution of protein sequences.
## This distribution is defined by probabilities of amino acid types at all positions.
## p[i,j,:] is a vector of 21 probablities represent the probablity of 21 amino acid type
## at the j'th position of the i'th sequence, so np.sum(p[i,j,:]) = 1
assert(np.all(np.sum(p, -1) - 1 <= 1e-6))

## the mapping between positions and amino acid types are stored in aa_index
with open("./output/aa_index.pkl", 'rb') as file_handle:
    aa_index = pickle.load(file_handle)
idx_to_aa_dict = {}
idx_to_aa_list = ['' for i in range(num_aa_type)]
for k, v in aa_index.items():
    idx_to_aa_dict[v] = k
    idx_to_aa_list[v] = k

## we can convert probablities into actual sequences by choosing the most likely sequences.
max_prob_idx = np.argmax(p, -1)
seqs = []
for i in range(num_seqs):
    seq = [idx_to_aa_list[idx] for idx in max_prob_idx[i,:]]
    seqs.append("".join(seq))

## the converted most likely sequences are in seqs
'''
for i in range(num_seqs):
    print(seqs[i])
    '''


## we can also get the second, third most likely sequences from p for each point in the latent space.

## YYY edited
print(len(seqs))
# save projected sequences
with open('./output/projected_sequence.fasta','w') as seq_handle:
    seq_handle.write(''.join(['>%s\n%s\n'%(x,y) for x,y in zip(range(len(seqs)),seqs)]))


## make_chimera_sequences.py


In [ ]:
__author__ = "Xinqiang Ding <xqding@umich.edu>"
__date__ = "2018/06/07 20:22:03"

import numpy as np
from sys import exit
import pickle

## parent sequences of CYP102A1, CYP102A2, CYP102A3
parent_sequences = []
with open("./chimera_sequence/parent_sequences.txt", 'r') as file_handle:
    for line in file_handle:
        line = line.strip()
        fields = line.split()
        parent_sequences.append(fields[1])

## each parent sequence is seperated into 8 segments        
positions = [64, 122, 166, 216, 268, 328, 404]
align_positions = [] ## correspondings position in the alignment
k = 0
for i in range(len(parent_sequences[0])):
    if parent_sequences[0][i] != ".":
        k += 1
    if k in positions:
        align_positions.append(i)
        
## cut parent sequences into 8 segments
segments = []
for k in range(3):
    segments.append([])
    start = 0
    end = align_positions[0]+1
    for i in range(0, len(align_positions)+1):
        if i == 0:
            start = 0
        else:
            start = align_positions[i-1] + 1

        if i >= len(align_positions):
            segments[-1].append(parent_sequences[k][start:])
        else:
            end = align_positions[i] + 1
            segments[-1].append(parent_sequences[k][start:end])

## remove gaps in sequences            
for i in range(len(segments)):
    for j in range(len(segments[i])):
        segments[i][j] = "".join([s for s in segments[i][j] if s != "."])

## generate all possible name        
def generate_name(str_list):    
    if len(str_list[0]) == 8:
        return str_list
    else:
        new_list = []    
        for s in str_list:
            for i in ['1', '2', '3']:
                new_list.append(s + i)
        return generate_name(new_list)
names = generate_name(['1', '2', '3'])

## paste segments based on its name to make chimera sequences
chimera_sequences = {}
for name in names:
    seq = ""
    for i in range(len(name)):
        seq += segments[int(name[i])-1][i]
    chimera_sequences[name] = seq

## save chimera sequences    
with open("./output/chimera_sequences.pkl", 'wb') as file_handle:
    pickle.dump(chimera_sequences, file_handle)

with open("./output/chimera_sequence.fasta", 'w') as file_handle:
    for key, seq in chimera_sequences.items():
        print("> " + key, sep = "", file = file_handle)
        print(seq, file = file_handle)


In [ ]:
%%bash

cat ./output/query_seq.fasta \
    ./output/chimera_sequence.fasta \
    ./chimera_sequence/other_sequences.fasta > ./output/T50_sequences.fasta


In [ ]:
drive.flush_and_unmount()

In [ ]:
%%bash -s "$pfam_id" "$gs"
zip -r $1\_results_arnold_1.zip *

In [ ]:
drive.mount('./drive')

In [ ]:
%%bash -s "$pfam_id" "$gs"
mv $1\_results_arnold_1.zip ./drive/My\ Drive

In [ ]:
drive.flush_and_unmount()